<a href="https://colab.research.google.com/github/notmanan/Depression-Detection-Through-Multi-Modal-Data/blob/master/CNN_Audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import gc

In [0]:


import numpy as np
import pandas as pd 
import sklearn

def makedata(X):
    for i in range(X.shape[0]):
        if(X[i,1] == 0):
            X[i,0] = 0
            for j in range(7):
                X[i,j+1] = 0
    # X = np.array(X)
    return X

dev = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/dev_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
test = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/full_test_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
train = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/train_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
 

def scale_down(X):
  if(X.shape[0]<70000):
    return X
  X_new = []
  for i in range(int(X.shape[0]/2)):
    cur_row = X[i*2]
    for j in range(1,2):
      if(i+j < X.shape[0]):
        cur_row += X[i+j]
    cur_row = cur_row/2.0
    X_new.append(cur_row)
  X_new = np.array(X_new)
  return X_new

def decrease_size(X):
  
  size = 30000
  if(X.shape[0]<size):
    dif = size - X.shape[0] 
    temp = np.zeros((dif,X.shape[1]))
    X = np.concatenate((X,temp),axis = 0)
  if(X.shape[0]>size):
    # dif = X.shape[0] - size
    # while(dif>0):
    #   index = np.random.randint(0,X.shape[0]-1)
    #   X[index] = (X[index]+ X[index+1])/2
    #   X = np.delete(X,index+1,axis = 0)
    #   dif -= 1
    X = X[:30000]
  return X





X_train = []
Y_train = []

X_test = []
Y_test = []
size = 40000

for i in range(len(test)):
    data = pd.read_csv('/content/drive/My Drive/MCA Dataset/test_data/'+str(int(test[i][0]))+"_COVAREP.csv",header = None)
    X_temp = data.iloc[:,:].values
    X_temp = makedata(X_temp)
    # X_temp = scale_down(X_temp)
    # X_temp = decrease_size(X_temp)
    X_temp = X_temp[X_temp.shape[0]-size:]
    X_test.append(X_temp)
    Y_test.append(test[i][1])
X_test = np.array(X_test)
print(X_test.shape)

Y_test = np.array(Y_test)
print(Y_test.shape)
      
test = []
gc.collect()


for i in range(len(train)):
    data = pd.read_csv('/content/drive/My Drive/MCA Dataset/train_data/'+str(int(train[i][0]))+"_COVAREP.csv",header = None)
    X_temp = data.iloc[:,:].values
    X_temp = makedata(X_temp)
    # X_temp = scale_down(X_temp)
    # X_temp = decrease_size(X_temp)
    X_temp = X_temp[X_temp.shape[0]-size:]
    X_train.append(X_temp)
    Y_train.append(train[i][1])

train = []
gc.collect()

for i in range(len(dev)):
    data = pd.read_csv('/content/drive/My Drive/MCA Dataset/dev_data/'+str(int(dev[i][0]))+"_COVAREP.csv",header = None)
    X_temp = data.iloc[:,:].values
    X_temp = makedata(X_temp)
    # X_temp = scale_down(X_temp)
    # X_temp = decrease_size(X_temp)
    X_temp = X_temp[X_temp.shape[0]-size:]
    X_train.append(X_temp)
    Y_train.append(dev[i][1])

X_temp = []
dev = []
gc.collect()


X_train = np.array(X_train, dtype = object)
Y_train = np.array(Y_train)
# print(Y_train.shape)
print(X_train.shape)
print(Y_train.shape)

# X_train = sklearn.preprocessing.normalize(X_train)
# X_test = sklearn.preprocessing.normalize(X_test)


(41, 40000, 74)
(41,)
(137, 40000, 74)
(137,)


In [0]:

# print(np.array([1,X_train[0]]).shape)
def upsample(X_train,Y_train):
  X_train_0 = X_train[Y_train==0]
  X_train_1 = X_train[Y_train==1]

  Y_train_1 = Y_train[Y_train==1]
  # print(Y_train_1.shape)
  # print(X_train_1.shape)
  size = X_train_0.shape[0] - X_train_1.shape[0]
  X = []
  Y = []
  X_train = list(X_train)
  Y_train = list(Y_train)
  while(size>0):
    size -= 1
    index = np.random.randint(0,X_train_1.shape[0]-1)
    leave_index = np.random.randint(0,len(X_train)-1)
    X_add = X_train_1[index]
    X_leave = X_train[leave_index]

    Y_add = Y_train_1[index]
    Y_leave = Y_train[leave_index]

    X_train[leave_index] = X_add
    X_train.append(X_leave)

    Y_train[leave_index] = Y_add
    Y_train.append(Y_leave)


  X_train = np.array(X_train)
  Y_train = np.array(Y_train)
  return X_train,Y_train

X_upsample, Y_upsample = upsample(X_train,Y_train)
print(X_upsample.shape)
print(Y_upsample.shape)
X_train = []
Y_train = []
gc.collect()


(196, 40000, 74)
(196,)


41

In [0]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout

# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv1D(60, 10, input_shape = (40000, 74), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling1D(pool_size = 3))


# classifier.add(Conv1D(50, 3, activation = 'relu'))
# classifier.add(MaxPooling1D(pool_size = 5))

# Adding a second convolutional layer
classifier.add(Conv1D(30, 5, activation = 'relu'))
classifier.add(MaxPooling1D(pool_size = 3))


classifier.add(Conv1D(15, 5, activation = 'relu'))
classifier.add(MaxPooling1D(pool_size = 3))

# Step 3 - Flattening
classifier.add(Flatten())
classifier.add(Dropout(0.5))
# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))
# classifier.add(Dense(units = 1, activation = 'sigmoid'))

from keras import metrics

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
class_weight = { 0: 0.3, 1:0.7}
classifier.fit(X_upsample, Y_upsample, epochs=10)
score = classifier.evaluate(X_test, Y_test)
from sklearn.metrics import classification_report

# print(classification_report(Y_train,classifier.predict(X_train)))
print(classification_report(Y_test,classifier.predict(X_test)))

Using TensorFlow backend.







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/10





196/196 [==============================] - 32s 165ms/step - loss: 7.3382 - acc: 0.5000
Epoch 2/10
196/196 [==============================] - 15s 78ms/step - loss: 7.9712 - acc: 0.5000
Epoch 3/10
196/196 [==============================] - 16s 79ms/step - loss: 7.9712 - acc: 0.5000
Epoch 4/10
196/196 [==============================] - 15s 78ms/step - loss: 7.9712 - acc: 0.5000
Epoch 5/10
196/196 [==============================] - 16s 79ms/step - loss: 7.9712 - acc: 0.5000
Epoch 6/10
196/196 [==============================] - 15s 79ms/step - loss: 7.9712 - acc: 0.5000
Epoch 7/10
196/196 [==============================] - 15s 78ms/step - loss: 7.9712 - acc: 0.5000
Epoch 8/10
196/196 [==============================] - 15s 77ms/step - loss: 7.9712 - acc: 0.5000

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
classifier.predict(X_test)

In [0]:
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv1D(32, 3, input_shape = (6000, 74), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling1D(pool_size = 2))

# Adding a second convolutional layer
classifier.add(Conv1D(32, 3, activation = 'relu'))
classifier.add(MaxPooling1D(pool_size = 2))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [0]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(137, 6000, 74)
(137,)
(41, 6000, 74)
(41,)


In [0]:
print(X_test.shape)

(10, 60000, 74)


In [0]:
print(X_train.shape)

(20,)


In [0]:
classifier.fit(np.array([X_train]), Y_train, epochs=2)
score = classifier.evaluate([X_test], Y_test)